In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.utils import to_categorical
import os

In [8]:
os.listdir()

['extract_files.ipynb',
 'test',
 'test.txt',
 'test.zip',
 'train',
 'train.txt',
 'train.zip',
 'try.ipynb',
 'valid',
 'valid.txt',
 'valid.zip']

In [14]:
train_dir = 'train'
validation_dir = 'valid'

In [10]:
# Define the base model using pre-trained weights of VGG16
base_model = applications.VGG16(weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 13s 0us/step


In [11]:
# Freeze the base model layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

In [12]:
# Add new layers for fine-tuning
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(256, activation='relu')(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(14, activation='softmax')(top_model)

# Combine the base model and the fine-tuned layers
model = tf.keras.Model(inputs=base_model.input, outputs=top_model)

In [15]:
# Create ImageDataGenerator objects for training and validation data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 260915 images belonging to 14 classes.
Found 47531 images belonging to 14 classes.


In [18]:
# Convert labels to categorical format
train_generator.labels = to_categorical(train_generator.labels)
validation_generator.labels = to_categorical(validation_generator.labels)

# Recompile the model with the updated output layer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# Train the model using the training and validation data generators
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(validation_generator)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "c:\Users\aditr\AppData\Local\Programs\Python\Python310\lib